# 부록 10.4.3: 차트, 그래프 및 슬라이드 데크 작업
Claude는 차트, 그래프 및 더 넓은 슬라이드 데크를 작업하는 데 매우 능숙합니다. 사용 사례에 따라 활용할 수 있는 다양한 팁과 요령이 있습니다. 이 레시피에서는 이러한 자료와 함께 Claude를 사용하는 일반적인 패턴을 보여줍니다.

## 차트 및 그래프
대부분의 경우 Claude를 차트와 그래프에 사용하는 것은 간단합니다. 이들을 수용하고 Claude에 전달하는 방법과 결과를 개선하기 위한 일반적인 팁을 살펴보겠습니다.

### 수용 및 Claude API 호출
Claude에 차트와 그래프를 전달하는 가장 좋은 방법은 비전 기능을 활용하는 것입니다. 즉, 차트나 그래프의 이미지와 함께 이에 대한 텍스트 질문을 Claude에게 제공합니다. 모든 버전의 Claude가 이미지를 수용할 수 있지만, Sonnet과 Opus는 데이터 중심 이미지 작업에 권장되는 모델입니다. Sonnet을 사용하여 시작해 보겠습니다.

In [ ]:
pip install -qUr requirements.txt

In [ ]:
import boto3
import json
import base64
from datetime import datetime
from IPython.display import Image
from botocore.exceptions import ClientError

session = boto3.Session()
region = session.region_name

#modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
modelId = 'anthropic.claude-3-haiku-20240307-v1:0'

print(f'Using modelId: {modelId}')
print('Using region: ', region)

bedrock_client = boto3.client(service_name = 'bedrock-runtime', region_name = region,)

In [ ]:
def get_completion(messages):
    converse_api_params = {
        "modelId": modelId,
        "messages": messages,
    }
    response = bedrock_client.converse(**converse_api_params)
    # Extract the generated text content from the response
    return response['output']['message']['content'][0]['text']

In [ ]:
# To start, we'll need an image. We will be using the .png image located at cvna_2021_annual_report_image.png.
# Start by reading in the image and encoding it as base64.
with open("./images/reading_charts_graphs/cvna_2021_annual_report_image.png", "rb") as f:
    image_file = f.read()

# Let's also see the image for ourself
Image(filename='./images/reading_charts_graphs/cvna_2021_annual_report_image.png') 

이 이미지를 간단한 질문과 함께 모델에 전달하는 방법을 살펴보겠습니다.

In [ ]:
messages = [
    {
        "role": 'user',
        "content": [
            {"text": "What's in this image? Answer in a single sentence."},
            {"image": {
                "format": 'png',
                "source": {"bytes": image_file }
                },
            }
        ]
    }
]

print(get_completion(messages))

꽤 괜찮네요! 이제 더 유용한 질문을 해봅시다.

In [ ]:
questions = [
    "What was CVNA revenue in 2020?",
    "How many additional markets has Carvana added since 2014?",
    "What was 2016 revenue per retail unit sold?"
]

for index, question in enumerate(questions):
    messages = [
        {
            "role": 'user',
            "content": [
                {"text": "What's in this image? Answer in a single sentence."},
                {"image": {
                    "format": 'png',
                    "source": {"bytes": image_file }
                    },
                }
            ]
        }
    ]
    
    print(f"\n----------Question {index+1}----------")
    print(get_completion(messages))

보시다시피 Claude는 차트와 그래프에 대해 상당히 구체적인 질문에 답할 수 있습니다. 그러나 이를 최대한 활용하기 위한 팁과 요령이 있습니다.
- 때때로 Claude의 산술 능력이 방해가 됩니다. 위의 세 번째 질문을 샘플링하면 산술 계산을 잘못하여 최종 답변이 잘못된 경우가 있음을 알 수 있습니다. Claude에게 계산기 도구를 제공하여 이러한 실수를 방지하는 것이 좋습니다.
- 매우 복잡한 차트와 그래프의 경우, Claude에게 "먼저 이미지에서 보이는 모든 데이터 포인트를 설명하라"고 요청하면 전통적인 사고 연쇄(Chain of Thought)에서 본 것과 유사한 개선 효과를 얻을 수 있습니다.
- Claude는 많은 그룹이 있는 그룹화된 막대 차트와 같이 색상에 많이 의존하는 차트에서 어려움을 겪을 수 있습니다. Claude에게 먼저 그래프의 색상을 16진수 코드로 식별하도록 요청하면 정확도가 높아집니다.

## 슬라이드 데크
이제 Claude가 차트와 그래프의 마법사임을 알았으니, 차트와 그래프의 진정한 고향인 슬라이드 데크로 확장하는 것이 논리적입니다!

슬라이드는 금융 서비스를 포함한 많은 분야에서 중요한 정보 원천입니다. PyPDF와 같은 패키지를 사용하여 슬라이드 데크에서 텍스트를 추출할 수는 있지만, 차트/그래프가 많은 특성상 이는 좋은 선택이 아닐 수 있습니다. 모델이 실제로 필요한 정보에 접근하기 어려울 수 있기 때문입니다. 결과적으로 비전이 훌륭한 대체재가 될 수 있습니다. 이 섹션에서는 비전 Claude를 사용하여 슬라이드 데크를 검토하는 방법과 이 접근 방식의 일반적인 문제점을 해결하는 방법에 대해 살펴보겠습니다.

일반적인 슬라이드 데크를 Claude에게 전달하는 가장 좋은 방법은 PDF로 다운로드한 다음 각 PDF 페이지를 이미지로 변환하는 것입니다. 다음과 같이 수행할 수 있습니다.

In [ ]:
#%pip install PyMuPDF

In [ ]:
from PIL import Image
import io
import fitz

def pdf_to_pngs(pdf_path, quality=75, max_size=(1024, 1024)):
    """
    Converts a PDF file to a list of PNG images.

    Args:
        pdf_path (str): The path to the PDF file.
        quality (int, optional): The quality of the output PNG images (default is 75).
        max_size (tuple, optional): The maximum size of the output images (default is (1024, 1024)).

    Returns:
        list: A list of PNG images as bytes.
    """
    # Open the PDF file
    doc = fitz.open(pdf_path)
    pdf_to_png_images = []

    # Iterate through each page of the PDF
    for page_num in range(doc.page_count):
        # Load the page
        page = doc.load_page(page_num)

        # Render the page as a PNG image
        pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))

        # Save the PNG image
        output_path = f"./images/reading_charts_graphs/slides/page_{page_num+1}.png"
        pix.save(output_path)

        # Open the saved image using PIL
        image = Image.open(output_path)

        # Resize the image if it exceeds the maximum size
        if image.size[0] > max_size[0] or image.size[1] > max_size[1]:
            image.thumbnail(max_size, Image.Resampling.LANCZOS)

        # Convert the PIL image to bytes
        image_data = io.BytesIO()
        image.save(image_data, format='PNG', optimize=True, quality=quality)
        image_data.seek(0)
        pdf_to_png_image = image_data.getvalue()

        # Append the PNG image bytes to the list
        pdf_to_png_images.append(pdf_to_png_image)

    # Close the PDF document
    doc.close()

    return pdf_to_png_images

# Specify the path to the PDF file
pdf_path = './images/reading_charts_graphs/twilio_q4_2023.pdf'
# Call the pdf_to_pngs function to convert the PDF to PNG images
pdf_pngs = pdf_to_pngs(pdf_path)

In [ ]:
# Now let's pass the first 20 of these images (in order) to Claude at once and ask it a question about the deck. 
# Why 20? Currently, the Anthropic API only allows you to pass in a maximum of 20 images.
# While this number will likely increase over time, we have some helpful tips for how to manage it later in this recipe.

content = [{"image": {"format": 'png', "source": {"bytes": pdf_png}}} for pdf_png in pdf_pngs[:20]]

question = "What was Twilio y/y revenue growth for fiscal year 2023?"
#question = "What was the non-GAAP gross margin?"

# Append the question to our images
content.append({"text": question})

messages = [
    {
        "role": 'user',
        "content": content
    }
]

print(get_completion(messages))

이 접근 방식은 시작하기에 좋은 방법이며, 일부 사용 사례에서는 최고의 성능을 제공합니다. 그러나 몇 가지 제한 사항이 있습니다.
- 최대 20개의 이미지만 포함할 수 있습니다(시간이 지나면서 이 제한을 늘릴 예정입니다)
- RAG의 일부로 슬라이드 콘텐츠를 사용하는 경우 이미지를 임베딩에 도입하면 문제가 발생할 수 있습니다

다행히도 Claude의 비전 기능을 활용하여 일반적인 PDF 전사보다 훨씬 더 높은 품질의 슬라이드 덱 **텍스트 형식** 표현을 얻을 수 있습니다.

이를 수행하는 가장 좋은 방법은 Claude에게 시작부터 끝까지 순차적으로 덱을 설명하도록 요청하고, 현재 슬라이드와 이전 설명을 전달하는 것입니다. 어떻게 하는지 살펴보겠습니다.

In [ ]:
# Define two functions that allow us to craft prompts for narrating our slide deck. We would adjut these prompts based on the nature of the deck, but keep the structure largely the same.
def build_previous_slides_prompt(previous_slide_narratives):
    prompt = '\n'.join([f"<slide_narration id={index+1}>\n{narrative}\n</slide_narration>" for index, narrative in enumerate(previous_slide_narratives)])
    return prompt

def build_slides_narration_prompt(previous_slide_narratives):
    if len(previous_slide_narratives) == 0:
        prompt = """You are the Twilio CFO, narrating your Q4 2023 earnings presentation.

You are currently on slide 1, shown in the image.
Please narrate this page from Twilio's Q4 2023 Earnings Presentation as if you were the presenter. Do not talk about any things, especially acronyms, if you are not exactly sure you know what they mean. Do not discuss anything not explicitly seen on this slide as there are more slides to narrate later that will likely cover that material.
Do not leave any details un-narrated as some of your viewers are vision-impaired, so if you don't narrate every number they won't know the number.

Put your narration in <narration> tags."""

    else:
        prompt = f"""You are the Twilio CFO, narrating your Q4 2023 earnings presentation. So far, here is your narration from previous slides:
<previous_slide_narrations>
{build_previous_slides_prompt(previous_slide_narratives)}
</previous_slide_narrations>

You are currently on slide {len(previous_slide_narratives)+1}, shown in the image.
Please narrate this page from Twilio's Q4 2023 Earnings Presentation as if you were the presenter, accounting for what you have already said on previous slides. Do not talk about any things, especially acronyms, if you are not exactly sure you know what they mean. Do not discuss anything not explicitly seen on this slide as there are more slides to narrate later that will likely cover that material.
Do not leave any details un-narrated as some of your viewers are vision-impaired, so if you don't narrate every number they won't know the number.

Use excruciating detail.

Put your narration in <narration> tags."""
    
    return prompt

In [ ]:
# Now we use our functions to narrate the entire deck. Note that this may take a few minutes to run (often up to 10).
import re
from tqdm import tqdm
previous_slide_narratives = []
for i, pdf_png in tqdm(enumerate(pdf_pngs)):
    messages = [
        {
            "role": 'user',
            "content": [
                {"text": build_slides_narration_prompt(previous_slide_narratives)},
                {"image": {
                    "format": 'jpeg',
                    "source": {"bytes": pdf_png }
                    },
                }
            ]
        }
    ]
    completion = get_completion(messages)


    pattern = r"<narration>(.*?)</narration>"
    match = re.search(pattern, completion.strip(), re.DOTALL)
    if match:
        narration = match.group(1)
    else:
        raise ValueError("No narration available.")
    
    previous_slide_narratives.append(narration)
    # If you want to see the narration we produced, uncomment the below line
    # print(narration)

slide_narration = build_previous_slides_prompt(previous_slide_narratives)

이제 텍스트 기반 설명(완벽하지는 않지만 꽤 괜찮습니다)을 가지고 있으므로 벡터 검색을 포함하여 모든 텍스트 전용 워크플로우에서 이 덱을 사용할 수 있습니다.

마지막으로 설명만으로 구성된 설정에 몇 가지 질문을 해보겠습니다!

In [ ]:
questions = [
    "What percentage of q4 total revenue was the Segment business line?",
    "Has the rate of growth of quarterly revenue been increasing or decreasing? Give just an answer.",
    "What was acquisition revenue for the year ended december 31, 2023 (including negative revenues)?"
]

for index, question in enumerate(questions):
    prompt = f"""You are an expert financial analyst analyzing a transcript of Twilio's earnings call.
Here is the transcript:
<transcript>
{slide_narration}
</transcript>

Please answer the following question:
<question>
{question}
</question>"""
    messages = [
        {
            "role": 'user',
            "content": [
                {"text": prompt},
            ]
        }
    ]

    print(f"\n----------Question {index+1}----------")
    print(get_completion(messages))

좋아 보입니다! 이러한 기술을 활용하면 슬라이드 덱과 같은 차트 및 그래프가 많이 포함된 콘텐츠에 모델을 적용할 준비가 되었습니다.